In [29]:
import matplotlib.pyplot as plt
import numpy as np
from simsopt._core import load
from simsopt.geo import plot, MeanSquaredCurvature, CurveSurfaceDistance, CurveCurveDistance, LpCurveCurvature
from simsopt.geo import SurfaceRZFourier, create_equally_spaced_curves, \
    CurveLength, curves_to_vtk, CurveCWSFourier, ArclengthVariation
from simsopt.field import Current, coils_via_symmetries, BiotSavart
from simsopt.objectives import SquaredFlux, QuadraticPenalty
from scipy.optimize import minimize

res = load(f'serial0021326.json')

MAXITER = 200
quadpoints = 300
USE_PRESIM = False
USE_CWS = True

major_R = 1
minor_r = 0.5 #parameter search
ncoils = 3
order = 3
nfp_coils = 1
current_0  = 1
current_mult = 1e5
FIX_ONE_CURRENT = True



nphi = 48
ntheta = 48
s = SurfaceRZFourier.from_vmec_input('input', range="full torus", nphi=nphi, ntheta=ntheta)

##creating coils and currents:
#if USE_PRESIM:
#    coils_presim = res[1]
#    surface = res[0][-1]
#    base_curves = [x.curve for x in coils_presim]
#    base_currents = [x.current for x in coils_presim]
#    base_currents[0].fix_all()
#else:
#    base_curves = create_equally_spaced_curves(ncoils, s.nfp, stellsym=True, R0=1, R1=0.6, order=order)
#    base_currents = [Current(current_0) for i in range(ncoils)]
#    if FIX_ONE_CURRENT:
#        base_currents[0].fix_all()

if USE_CWS:
    theta_linspace = np.linspace(0, 1, ntheta, endpoint=True)
    phi_linspace = np.linspace(0, 1, nphi, endpoint=True)
    # currently don't know what this does, most likely only extension away from plasma
    # this could be necessary if the CWS is more complicated and is generated from a stage I 
    # ext_via_normal_factor = 0.2565
    #ext_via_normal_factor = 0.1482   
    #cws.extend_via_normal(ext_via_normal_factor)
    #0.25216216216216214
    # CREATE COIL WINDING SURFACE SURFACE
    cws = SurfaceRZFourier(nfp=nfp_coils,stellsym=True)
    cws.set_rc(0,0,major_R)
    cws.set_rc(major_R,0,minor_r+0.1)
    cws.set_zs(major_R,0,minor_r+0.1)

    # CREATE CURVES + COILS     
    base_curves = []
    for i in range(ncoils):
        curve_cws = CurveCWSFourier(
            mpol=cws.mpol,
            ntor=cws.ntor,
            idofs=cws.x,
            quadpoints=quadpoints,
            order=order,
            nfp=nfp_coils,
            stellsym=cws.stellsym,
        )
        angle = (i + 0.5)*(2 * np.pi)/((2) * s.nfp * ncoils)
        curve_dofs = np.zeros(len(curve_cws.get_dofs()),)
        curve_dofs[0] = 1
        curve_dofs[2*order+2] = 0
        curve_dofs[2*order+3] = angle
        curve_cws.set_dofs(curve_dofs)
        curve_cws.fix(0)
        curve_cws.fix(2*order+2)
        base_curves.append(curve_cws)
    base_currents = [Current(current_0)*current_mult for i in range(ncoils)]
    if FIX_ONE_CURRENT:
        base_currents[0].fix_all()
   


coils = coils_via_symmetries(base_curves, base_currents, nfp_coils*2, True)
curves = [c.curve for c in coils]

fig = plot(coils + [s] , engine="plotly", close=True, show=False)
fig.update_layout(
    autosize=False,
    width=1024,
    height=1024,)
fig.show()

bs = BiotSavart(coils)
bs.set_points(s.gamma().reshape((-1, 3)))

B_dot_n = np.sum(bs.B().reshape((nphi, ntheta, 3)) * s.unitnormal(), axis=2)
print('Initial max|B dot n|:', np.max(np.abs(B_dot_n)))


Initial max|B dot n|: 0.26125535217141904


In [30]:
##calculating magnetic field from coils:

# another init values from repo
# LENGTH_THRESHOLD = 20
# LENGTH_WEIGHT = 1e-8
# CC_THRESHOLD = 0.1
# CC_WEIGHT = 100
# CURVATURE_THRESHOLD = 60
# CURVATURE_WEIGHT = 1e-5
# MSC_THRESHOLD = 20
# MSC_WEIGHT = 1e-9
# ARCLENGTH_WEIGHT = 3e-8
# LENGTH_CON_WEIGHT = 0.1
# MAXITER = 50


FLUX_WEIGHT = 1

##setting up parameters for penalty function:
# Threshold and weight for the maximum length of each individual coil:
#LENGTH_THRESHOLD = 20
#LENGTH_WEIGHT = 1e-8

LENGTH_THRESHOLD = 20  
LENGTH_WEIGHT = 1e-8


# Threshold and weight for the coil-to-coil distance penalty in the objective function:
#CC_THRESHOLD = 0.1
#CC_WEIGHT = 100

CC_THRESHOLD = 0.2
CC_WEIGHT = 100


# Threshold and weight for the curvature penalty in the objective function:
#CURVATURE_THRESHOLD = 60
#CURVATURE_WEIGHT = 1e-5

CURVATURE_THRESHOLD = 20
CURVATURE_WEIGHT = 1e-2


# Threshold and weight for the mean squared curvature penalty in the objective function:
#MSC_THRESHOLD = 60
#MSC_WEIGHT = 1e-9
#ARCLENGTH_WEIGHT = 3e-8
#LENGTH_CON_WEIGHT = 0.1

MSC_THRESHOLD = 60
MSC_WEIGHT = 1e-9
ARCLENGTH_WEIGHT = 3e-8
LENGTH_CON_WEIGHT = 0.1

LENGTH_THRESHOLD = 20
LENGTH_WEIGHT = 1e-8
CC_THRESHOLD = 0.1
CC_WEIGHT = 100
CURVATURE_THRESHOLD = 60
CURVATURE_WEIGHT = 1e-5
MSC_THRESHOLD = 20
MSC_WEIGHT = 1e-9
ARCLENGTH_WEIGHT = 3e-8
LENGTH_CON_WEIGHT = 0.1

Jf = FLUX_WEIGHT * SquaredFlux(s, bs, definition="local")
Jls = [CurveLength(c) for c in base_curves]
Jccdist = CurveCurveDistance(curves, CC_THRESHOLD, num_basecurves=len(curves))
Jcs = [LpCurveCurvature(c, 2, CURVATURE_THRESHOLD) for i, c in enumerate(base_curves)]
Jmscs = [MeanSquaredCurvature(c) for c in base_curves]
Jals = [ArclengthVariation(c) for c in base_curves]
J_LENGTH = LENGTH_WEIGHT * sum(Jls)
J_CC = CC_WEIGHT * Jccdist
J_CURVATURE = CURVATURE_WEIGHT * sum(Jcs)
J_MSC = MSC_WEIGHT * sum(QuadraticPenalty(J, MSC_THRESHOLD, f="max") for i, J in enumerate(Jmscs))
J_ALS = ARCLENGTH_WEIGHT * sum(Jals)
J_LENGTH_PENALTY = LENGTH_CON_WEIGHT * sum([QuadraticPenalty(Jls[i], LENGTH_THRESHOLD, f="max") for i in range(len(base_curves))])
JF = Jf + J_CC + J_LENGTH_PENALTY + J_CURVATURE + J_ALS + J_MSC + J_LENGTH


B_dot_n = np.sum(bs.B().reshape((nphi, ntheta, 3)) * s.unitnormal(), axis=2)

print('Initial max|B dot n|:', np.max(np.abs(B_dot_n)))



def fun(dofs):
    JF.x = dofs
    J = JF.J()
    grad = JF.dJ()
    jf = Jf.J()
    BdotN = np.mean(np.abs(np.sum(bs.B().reshape((nphi, ntheta, 3)) * s.unitnormal(), axis=2)))
    outstr = f"J={J:.3e}, Jf={jf:.3e}, ⟨B·n⟩={BdotN:.1e}"
    cl_string = ", ".join([f"{J.J():.1f}" for J in Jls])
    kap_string = ", ".join(f"{np.max(c.kappa()):.1f}" for c in base_curves)
    msc_string = ", ".join(f"{J.J():.1f}" for J in Jmscs)
    outstr += f", Len=sum([{cl_string}])={sum(J.J() for J in Jls):.1f}, ϰ=[{kap_string}], ∫ϰ²/L=[{msc_string}]"
    outstr += f", C-C-Sep={Jccdist.shortest_distance():.2f}"
    print(outstr)
    return J, grad
    

print("""
################################################################################
### Perform a Taylor test ######################################################
################################################################################
""")
f = fun
dofs = JF.x
np.random.seed(1)
h = np.random.uniform(size=dofs.shape)
J0, dJ0 = f(dofs)
dJh = sum(dJ0 * h)
for eps in [1e-3, 1e-4, 1e-5, 1e-6, 1e-7]:
    J1, _ = f(dofs + eps*h)
    J2, _ = f(dofs - eps*h)
    print("err", (J1-J2)/(2*eps) - dJh)

print("""
################################################################################
### Run the optimisation #######################################################
################################################################################
""")

B_dot_n = np.sum(bs.B().reshape((nphi, ntheta, 3)) * s.unitnormal(), axis=2)
B_dot_n_max = np.max(np.abs(B_dot_n)).copy()

total_curve_length_start = sum(func.J() for func in Jls).copy()


res = minimize(fun, dofs, jac=True, method='L-BFGS-B',
               options={'maxiter': MAXITER, 'maxcor': 500, 'iprint': 5}, tol=1e-10)

B_dot_n = np.sum(bs.B().reshape((nphi, ntheta, 3)) * s.unitnormal(), axis=2)
print('starting max|B dot n|:', B_dot_n_max)
print('Final max|B dot n|:', np.max(np.abs(B_dot_n)))
total_curve_length = sum(func.J() for func in Jls)
print("starting Sum of lengths of base curves:", total_curve_length_start)
print("Sum of lengths of base curves:", total_curve_length)

Initial max|B dot n|: 0.26125535217141904

################################################################################
### Perform a Taylor test ######################################################
################################################################################

J=2.593e-01, Jf=2.593e-01, ⟨B·n⟩=4.7e-02, Len=sum([3.8, 3.8, 3.8])=11.3, ϰ=[1.7, 1.7, 1.7], ∫ϰ²/L=[2.8, 2.8, 2.8], C-C-Sep=0.21
J=2.594e-01, Jf=2.594e-01, ⟨B·n⟩=4.7e-02, Len=sum([3.8, 3.8, 3.8])=11.3, ϰ=[1.7, 1.7, 1.7], ∫ϰ²/L=[2.8, 2.8, 2.8], C-C-Sep=0.21
J=2.593e-01, Jf=2.593e-01, ⟨B·n⟩=4.7e-02, Len=sum([3.8, 3.8, 3.8])=11.3, ϰ=[1.7, 1.7, 1.7], ∫ϰ²/L=[2.8, 2.8, 2.8], C-C-Sep=0.21
err 2.1108307215189637e-06
J=2.593e-01, Jf=2.593e-01, ⟨B·n⟩=4.7e-02, Len=sum([3.8, 3.8, 3.8])=11.3, ϰ=[1.7, 1.7, 1.7], ∫ϰ²/L=[2.8, 2.8, 2.8], C-C-Sep=0.21
J=2.593e-01, Jf=2.593e-01, ⟨B·n⟩=4.7e-02, Len=sum([3.8, 3.8, 3.8])=11.3, ϰ=[1.7, 1.7, 1.7], ∫ϰ²/L=[2.8, 2.8, 2.8], C-C-Sep=0.21
err 2.1125812404987343e-08
J=2.593e-01, Jf=2.5

 This problem is unconstrained.


J=4.804e-01, Jf=2.555e-01, ⟨B·n⟩=4.7e-02, Len=sum([5.0, 4.8, 4.9])=14.7, ϰ=[8.3, 7.5, 5.3], ∫ϰ²/L=[6.2, 6.9, 5.7], C-C-Sep=0.00
J=1.287e-01, Jf=1.287e-01, ⟨B·n⟩=3.2e-02, Len=sum([4.1, 4.0, 4.0])=12.1, ϰ=[4.0, 3.7, 2.9], ∫ϰ²/L=[4.1, 4.1, 3.7], C-C-Sep=0.13
J=9.431e-02, Jf=8.987e-02, ⟨B·n⟩=2.5e-02, Len=sum([4.2, 4.4, 4.2])=12.8, ϰ=[5.1, 6.1, 4.0], ∫ϰ²/L=[4.7, 5.8, 4.8], C-C-Sep=0.07
J=1.158e-01, Jf=8.305e-02, ⟨B·n⟩=2.3e-02, Len=sum([4.2, 4.5, 4.4])=13.1, ϰ=[4.8, 7.3, 5.3], ∫ϰ²/L=[4.9, 6.3, 5.3], C-C-Sep=0.04
J=9.075e-02, Jf=8.658e-02, ⟨B·n⟩=2.4e-02, Len=sum([4.2, 4.4, 4.3])=12.9, ϰ=[4.9, 6.4, 4.4], ∫ϰ²/L=[4.7, 6.0, 4.9], C-C-Sep=0.07
J=8.876e-02, Jf=8.871e-02, ⟨B·n⟩=2.4e-02, Len=sum([4.1, 4.3, 4.2])=12.7, ϰ=[4.4, 6.0, 4.2], ∫ϰ²/L=[4.5, 5.7, 4.7], C-C-Sep=0.09
J=8.519e-02, Jf=8.374e-02, ⟨B·n⟩=2.3e-02, Len=sum([4.2, 4.4, 4.3])=12.9, ϰ=[4.4, 6.6, 4.8], ∫ϰ²/L=[4.5, 6.0, 5.0], C-C-Sep=0.08

At iterate    5    f=  8.51884D-02    |proj g|=  1.16824D-01
J=8.077e-02, Jf=7.795e-02, ⟨B·n⟩=2.2e-02, 


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.


J=4.123e-02, Jf=4.021e-02, ⟨B·n⟩=1.6e-02, Len=sum([4.4, 4.6, 5.5])=14.5, ϰ=[50.9, 34.3, 128.7], ∫ϰ²/L=[16.3, 20.9, 55.6], C-C-Sep=0.09
J=4.087e-02, Jf=4.015e-02, ⟨B·n⟩=1.6e-02, Len=sum([4.4, 4.6, 5.5])=14.5, ϰ=[55.6, 35.4, 121.5], ∫ϰ²/L=[17.7, 21.3, 53.6], C-C-Sep=0.09

At iterate   25    f=  4.08750D-02    |proj g|=  3.59186D-02
J=4.062e-02, Jf=4.002e-02, ⟨B·n⟩=1.6e-02, Len=sum([4.5, 4.6, 5.5])=14.5, ϰ=[62.8, 36.8, 112.8], ∫ϰ²/L=[19.6, 22.1, 50.8], C-C-Sep=0.09
J=4.038e-02, Jf=3.974e-02, ⟨B·n⟩=1.5e-02, Len=sum([4.5, 4.6, 5.5])=14.5, ϰ=[75.6, 40.0, 98.8], ∫ϰ²/L=[23.0, 23.5, 46.6], C-C-Sep=0.09
J=4.098e-02, Jf=3.923e-02, ⟨B·n⟩=1.5e-02, Len=sum([4.6, 4.6, 5.4])=14.6, ϰ=[101.1, 47.1, 71.9], ∫ϰ²/L=[30.1, 26.9, 40.2], C-C-Sep=0.08
J=4.026e-02, Jf=3.956e-02, ⟨B·n⟩=1.5e-02, Len=sum([4.5, 4.6, 5.4])=14.6, ϰ=[79.0, 42.3, 92.1], ∫ϰ²/L=[24.4, 24.3, 44.9], C-C-Sep=0.09
J=7.867e-02, Jf=3.891e-02, ⟨B·n⟩=1.6e-02, Len=sum([4.8, 4.6, 5.3])=14.8, ϰ=[143.9, 64.8, 46.7], ∫ϰ²/L=[43.1, 35.8, 32.3], C-C-Sep=

...

J=3.288e-02, Jf=3.194e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[49.2, 52.2, 50.9], ∫ϰ²/L=[26.6, 29.2, 33.7], C-C-Sep=0.09

At iterate  190    f=  3.28808D-02    |proj g|=  3.07613D-02
J=3.290e-02, Jf=3.194e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[48.8, 51.7, 50.9], ∫ϰ²/L=[26.6, 29.1, 33.7], C-C-Sep=0.09
J=3.288e-02, Jf=3.194e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[49.1, 52.1, 50.9], ∫ϰ²/L=[26.6, 29.2, 33.7], C-C-Sep=0.09
J=3.288e-02, Jf=3.194e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[49.2, 52.2, 50.9], ∫ϰ²/L=[26.6, 29.2, 33.7], C-C-Sep=0.09
J=3.288e-02, Jf=3.194e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[49.2, 52.2, 50.9], ∫ϰ²/L=[26.6, 29.2, 33.7], C-C-Sep=0.09
J=3.288e-02, Jf=3.194e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[49.2, 52.2, 50.9], ∫ϰ²/L=[26.6, 29.2, 33.7], C-C-Sep=0.09
J=3.288e-02, Jf=3.194e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[49.2, 52.2, 50.9], ∫ϰ²/L=[26.6, 29.2, 33.7], C-C-Sep=0.09



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.


J=5.686e-02, Jf=3.172e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.4, 4.8, 5.4])=15.6, ϰ=[58.3, 46.2, 51.1], ∫ϰ²/L=[28.7, 27.6, 33.7], C-C-Sep=0.06
J=3.291e-02, Jf=3.191e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[48.6, 51.8, 50.9], ∫ϰ²/L=[26.7, 29.1, 33.7], C-C-Sep=0.09
J=3.286e-02, Jf=3.193e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[49.0, 52.1, 50.9], ∫ϰ²/L=[26.6, 29.2, 33.7], C-C-Sep=0.09
J=3.291e-02, Jf=3.190e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[47.9, 51.2, 51.0], ∫ϰ²/L=[26.7, 29.0, 33.7], C-C-Sep=0.09
J=3.286e-02, Jf=3.192e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[48.8, 51.9, 50.9], ∫ϰ²/L=[26.6, 29.1, 33.7], C-C-Sep=0.09
J=3.288e-02, Jf=3.190e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[48.2, 51.5, 50.9], ∫ϰ²/L=[26.7, 29.0, 33.7], C-C-Sep=0.09
J=3.286e-02, Jf=3.192e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[48.7, 51.8, 50.9], ∫ϰ²/L=[26.6, 29.1, 33.7], C-C-Sep=0.09
J=3.317e-02, Jf=3.184e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.


J=3.286e-02, Jf=3.192e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[48.6, 51.7, 50.9], ∫ϰ²/L=[26.6, 29.1, 33.7], C-C-Sep=0.09
J=3.293e-02, Jf=3.192e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[47.8, 50.3, 51.1], ∫ϰ²/L=[26.7, 29.0, 33.7], C-C-Sep=0.09
J=3.286e-02, Jf=3.192e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[48.3, 51.4, 51.0], ∫ϰ²/L=[26.7, 29.1, 33.7], C-C-Sep=0.09
J=3.286e-02, Jf=3.192e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[48.5, 51.6, 50.9], ∫ϰ²/L=[26.6, 29.1, 33.7], C-C-Sep=0.09
J=3.286e-02, Jf=3.192e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[48.4, 51.5, 51.0], ∫ϰ²/L=[26.7, 29.1, 33.7], C-C-Sep=0.09
J=3.286e-02, Jf=3.192e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[48.5, 51.6, 50.9], ∫ϰ²/L=[26.7, 29.1, 33.7], C-C-Sep=0.09
J=3.286e-02, Jf=3.192e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5.4])=15.5, ϰ=[48.4, 51.5, 50.9], ∫ϰ²/L=[26.7, 29.1, 33.7], C-C-Sep=0.09
J=3.286e-02, Jf=3.192e-02, ⟨B·n⟩=1.4e-02, Len=sum([5.3, 4.8, 5

In [31]:
from simsopt._core import save

save(bs,'bs_opt.json')
save(bs.coils,'coils_opt.json')

In [32]:
#fig = plot(coils + [s] + [cws], engine="plotly", close=True, show=False)
fig = plot(coils + [s] , engine="plotly", close=True, show=False)
fig.update_layout(
    autosize=False,
    width=1024,
    height=1024,)
fig.show()

In [33]:
from simsopt.field.magneticfieldclasses import InterpolatedField
import os

def make_Bnormal_plots(bs, s_plot, out_dir='', bs_filename="Bnormal"):
    """
    Plot Bnormal on plasma surface from a MagneticField object.
    Do this quite a bit in the permanent magnet optimization
    and initialization so this is a wrapper function to reduce
    the amount of code.

    Args:
        bs: MagneticField class object.
        s_plot: plasma boundary surface with range = 'full torus'.
        out_dir: Path or string for the output directory for saved files.
        bs_filename: String denoting the name of the output file. 
    """
    nphi = len(s_plot.quadpoints_phi)
    ntheta = len(s_plot.quadpoints_theta)
    bs.set_points(s_plot.gamma().reshape((-1, 3)))
    pointData = {"B_N": np.sum(bs.B().reshape((nphi, ntheta, 3)) * s_plot.unitnormal(), axis=2)[:, :, None]}
    s_plot.to_vtk(out_dir + bs_filename, extra_data=pointData)

def trace_fieldlines(bfield, label, s, out_dir=''):
    """
    Make Poincare plots on a surface as in the trace_fieldlines
    example in the examples/1_Simple/ directory.

    Args:
        bfield: MagneticField or InterpolatedField class object.
        label: Name of the file to write to.
        s: plasma boundary surface.
        out_dir: Path or string for the output directory for saved files.
    """
    from simsopt.field.tracing import compute_fieldlines, \
        plot_poincare_data, \
        IterationStoppingCriterion,LevelsetStoppingCriterion, SurfaceClassifier

    # set fieldline tracer parameters
    nfieldlines = 32
    tmax_fl = 2500
    nplanes = 4

    Z0 = np.zeros(nfieldlines)
    #R0 = np.linspace(s.get_rc(0, 0), s.get_rc(0, 0) + s.get_rc(1, 0) / 4.0, nfieldlines)
    #R0 = np.linspace(s.get_rc(0, 0), s.get_rc(0, 0) + s.get_rc(1, 0) / 2.0, nfieldlines)
    R0 = np.linspace(0.9, 1.4, nfieldlines)
    #Z0 = np.zeros(nfieldlines)
    phis = [(i / 4) * (2 * np.pi / s.nfp) for i in range(nplanes)]

    print("Compute the fieldlines from the initial locations")
    sc_fieldline = SurfaceClassifier(s, h=0.03, p=2)
    #sc_fieldline.to_vtk(str(out_dir) + 'levelset', h=0.02)

    stopping_crit = [LevelsetStoppingCriterion(sc_fieldline.dist)]

    fieldlines_tys, fieldlines_phi_hits = compute_fieldlines(
        bfield, R0, Z0, tmax=tmax_fl, tol=1e-8,
        phis=phis,
        stopping_criteria=stopping_crit)

    print("Plotting")
    plot_poincare_data(fieldlines_phi_hits, phis, out_dir + label + '.png', dpi=100, surf=s)


# Directory for output
OUT_DIR = "./poincare/"
os.makedirs(OUT_DIR, exist_ok=True)

# calculating magnetic field from coils:
bs = BiotSavart(coils)


n = 64
rs = np.linalg.norm(s.gamma()[:, :, 0:2], axis=2)
zs = s.gamma()[:, :, 2]
r_margin = 0.05
rrange = (np.min(rs) - r_margin, np.max(rs) + r_margin, n)
phirange = (0, 2 * np.pi / s.nfp, n * 2)
zrange = (0, np.max(zs), n // 2)
degree = 4  # 2 is sufficient sometimes
nphi = len(s.quadpoints_phi)
ntheta = len(s.quadpoints_theta)
bs.set_points(s.gamma().reshape((-1, 3)))
Bnormal = np.sum(bs.B().reshape((nphi, ntheta, 3)) * s.unitnormal(), axis=2)
make_Bnormal_plots(bs, s, OUT_DIR, "biot_savart_pre_poincare_check")

print("Interpolating Field")
bsh = InterpolatedField(
    bs , degree, rrange, phirange, zrange, True, nfp=s.nfp, stellsym=s.stellsym
)
bsh.set_points(s.gamma().reshape((-1, 3)))

bsh.to_vtk('poincare/1_CWS_bsh')
trace_fieldlines(bsh, 'poincare_1_CWS', s, OUT_DIR)

Interpolating Field
Compute the fieldlines from the initial locations
Plotting


<Figure size 640x480 with 0 Axes>

In [34]:
bs.plot()

AttributeError: 'BiotSavart' object has no attribute 'plot'